In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf 
import tensorflow_hub as tfhub
import tensorflow_datasets as tfds

In [2]:
(raw_train, raw_validation, raw_test), metadata = tfds.load(
    'cats_vs_dogs',
    split=['train[:80%]', 'train[80%:90%]', 'train[90%:]'],
    with_info=True,
    as_supervised=True,
)

def format_image(image, label):
    image = tf.image.resize(image, (224, 224)) / 255.0
    return  image, label

BATCH_SIZE = 16
num_examples = metadata.splits['train'].num_examples
num_classes = metadata.features['label'].num_classes

train_batches = (raw_train.shuffle(num_examples // 4)
                 .map(format_image).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE))

validation_batches = (raw_validation
                      .map(format_image)
                      .batch(BATCH_SIZE)
                      .prefetch(tf.data.AUTOTUNE))

test_batches = raw_test.map(format_image).batch(BATCH_SIZE)


### Model

In [3]:
from tensorflow.keras.applications.resnet50 import ResNet50 

In [4]:
base_model = ResNet50(input_shape=(224,224,3),
                         include_top=False,pooling='avg')

# Customize the output layers
flatten_all = tf.keras.layers.Flatten()(base_model.output)
Dense_1 = tf.keras.layers.Dense(units=512,activation='relu')(flatten_all)
prediction_layer = tf.keras.layers.Dense(units=2, activation='softmax')(Dense_1)

# Concatenate the model
model = tf.keras.models.Model(inputs=base_model.input, outputs=prediction_layer)

model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

# hist = model.fit(train_batches,
#                  epochs=EPOCHS,
#                  validation_data=validation_batches)

# model.summary()

## 

## Quantization Aware Training with Keras Models

In [1]:
import tensorflow_model_optimization as tfmot

In [ ]:
tfmot.quantization.keras.quantiz

In [5]:
import tensorflow_model_optimization as tfmot

quantize_model = tfmot.quantization.keras.quantize_model

# q_aware stands for for quantization aware.
q_aware_model = quantize_model(model)

# `quantize_model` requires a recompile.
q_aware_model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

# q_aware_model.summary()
q_aware_model.fit(train_batches, validation_data=validation_batches, epochs=5,)

C:\Users\user\anaconda3\envs\tensorflow2\lib\site-packages\keras\layers\wrappers.py:64: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  config = {'layer': generic_utils.serialize_keras_object(self.layer)}
C:\Users\user\anaconda3\envs\tensorflow2\lib\site-packages\keras\engine\functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


Epoch 1/5
1164/1164 [==============================] - 862s 731ms/step - loss: 0.6818 - accuracy: 0.5825 - val_loss: 1.0418 - val_accuracy: 0.4815
Epoch 2/5
1164/1164 [==============================] - 850s 730ms/step - loss: 0.6928 - accuracy: 0.5697 - val_loss: 0.7201 - val_accuracy: 0.5778
Epoch 3/5
1164/1164 [==============================] - 849s 729ms/step - loss: 0.6521 - accuracy: 0.6008 - val_loss: 0.6899 - val_accuracy: 0.5520
Epoch 4/5
1164/1164 [==============================] - 849s 729ms/step - loss: 0.6447 - accuracy: 0.6155 - val_loss: 0.7505 - val_accuracy: 0.5451
Epoch 5/5
1164/1164 [==============================] - 856s 735ms/step - loss: 0.6263 - accuracy: 0.6358 - val_loss: 0.6567 - val_accuracy: 0.5993


In [7]:
tf.saved_model.save(q_aware_model,'Test/')

INFO:tensorflow:Assets written to: Test/assets


INFO:tensorflow:Assets written to: Test/assets


In [6]:
converter = tf.lite.TFLiteConverter.from_keras_model(q_aware_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

quantized_tflite_model = converter.convert()

INFO:tensorflow:Assets written to: C:\Users\user\AppData\Local\Temp\tmpgys1ymhk\assets


INFO:tensorflow:Assets written to: C:\Users\user\AppData\Local\Temp\tmpgys1ymhk\assets


In [9]:
with open('test.tflite','wb') as f:
    f.write(quantized_tflite_model)